In [1]:
import sys
import os
Lpath = os.path.abspath('..')
print("found path with os.path.abspath('..'): ", Lpath)
sys.path.insert(0, Lpath)

print(sys.path)

import numpy as np
import tflearn
import tflearn.activations as activations
# Data loading and preprocessing
from tflearn.activations import relu
from tflearn.data_utils import shuffle, to_categorical
from tflearn.layers.conv import avg_pool_2d, conv_2d, max_pool_2d, global_avg_pool 
from tflearn.layers.core import dropout, flatten, fully_connected, input_data
from tflearn.layers.merge_ops import merge
from tflearn.layers.normalization import local_response_normalization
from tflearn.layers.normalization import batch_normalization



#chris library imports
from matplotlib import pyplot as plt
import cv2
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize

from test_recursive_image_load_V2 import load_CKP_data
from test_recursive_image_load_V2 import load_formated_data
from test_recursive_image_load_V2 import split_dataset
from test_recursive_image_load_V2 import divide_subjects
from test_recursive_image_load_V2 import divide_data_to_subject
from test_recursive_image_load_V2 import load_npy_files

from showNumpyInfo import showInfo

# from Dexpression_network import create_Dexpression_old_network
# from Dexpression_network import create_Dexpression_network
# # from Dexpression_network import create_Dexpression_FX2_out_layer

from display_image import display


found path with os.path.abspath('..'):  G:\Documenten\personal\school\MaNaMA_AI\thesis\implementation\dexpression\github_1\github\Thesis_Christiaan_Jean\Custom_Dexpression
['G:\\Documenten\\personal\\school\\MaNaMA_AI\\thesis\\implementation\\dexpression\\github_1\\github\\Thesis_Christiaan_Jean\\Custom_Dexpression', '', 'a:\\users\\christiaan\\appdata\\local\\programs\\python\\python36\\python36.zip', 'a:\\users\\christiaan\\appdata\\local\\programs\\python\\python36\\DLLs', 'a:\\users\\christiaan\\appdata\\local\\programs\\python\\python36\\lib', 'a:\\users\\christiaan\\appdata\\local\\programs\\python\\python36', 'a:\\users\\christiaan\\appdata\\local\\programs\\python\\python36\\lib\\site-packages', 'a:\\users\\christiaan\\appdata\\local\\programs\\python\\python36\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\christiaan\\.ipython']
hdf5 is not supported on this machine (please install/reinstall h5py for optimal experience)
curses is not supported on this machine (please i

In [2]:
# global Paths to define for each specific computer
#tf_checkpoints = where the checkpoints of tensorflow training algorithms are stored to be recovered if necessary
tf_checkpoints = "G:/Documenten/personal/school/MaNaMA_AI/thesis/implementation/dexpression/github_1/github/Thesis_Christiaan_Jean/Custom_Dexpression/tf_checkpoints"

#load data
# try:
#     X_data = np.load('../data/CKP_X.npy')
#     Y_data = np.load('../data/CKP_Y.npy')
#     X_subID = (np.load('../data/CKP_subjectIds.npy')).astype('uint8')
# except:
#     X_data = np.load('G:/Documenten/personal/school/MaNaMA_AI/thesis/implementation/dexpression/github_1/github/Thesis_Christiaan_Jean/data/CKP_X.npy')
#     Y_data = np.load('G:/Documenten/personal/school/MaNaMA_AI/thesis/implementation/dexpression/github_1/github/Thesis_Christiaan_Jean/data/CKP_Y.npy')
#     X_subID = (np.load('G:/Documenten/personal/school/MaNaMA_AI/thesis/implementation/dexpression/github_1/github/Thesis_Christiaan_Jean/data/CKP_subjectIds.npy')).astype('uint8')
dire = 'CKP_all'

[X_data,Y_data,X_subID] = load_npy_files(5,dire)

#load the subject distribution over the different datasets
subID = (np.load('../data_division/'+dire+'/train_subject_ID.npy')).astype('uint8')
subID_val = (np.load('../data_division/'+dire+'/validation_subject_ID.npy')).astype('uint8')
subID_test = (np.load('../data_division/'+dire+'/test_subject_ID.npy')).astype('uint8')
subIDs = [subID, subID_val, subID_test]

# divided_data = divide_data_to_subject([X_data,Y_data,X_subID],subIDs)

# print(X.shape)
# print(X.dtype)

# def format_data(divided_data):
#     X = (divided_data[0].reshape(-1,224,224,1)).astype('uint8')
#     Y = (divided_data[1].reshape(-1,7)).astype('uint8')

#     # create the validation set X_val and Y-val (SubID_val is not given to the network)
#     X_val = divided_data[2].reshape(-1,224,224,1).astype('uint8')
#     Y_val = divided_data[3].reshape(-1,7).astype('uint8')

#     # create the test set X_test and Y_test (SubID_test is not given to the network)
#     X_test = divided_data[4].reshape(-1,224,224,1).astype('uint8')
#     Y_test = divided_data[5].reshape(-1,7).astype('uint8')  
    
#     return [X,Y,X_val,Y_val,X_test,Y_test]

# [X,Y,X_val,Y_val,X_test,Y_test] = format_data(divided_data)
# print(X.shape)
# print(X.dtype)
tf.reset_default_graph()
tflearn.config.init_training_mode()

[WARNING] could not load from path: ./data/CKP_all 
 because of: [Errno 2] No such file or directory: './data/CKP_all/X.npy' error
[WARNING] could not load from path: ./../data/CKP_all 
 because of: [Errno 2] No such file or directory: './../data/CKP_all/X.npy' error
[SUCCEED]data found in path:  ./../../data/CKP_all


In [3]:
    # Give a dropout if required (change to True and define the dropout percentage).
    dropout = False
    dropout_keep_prob = 0.5
    
    # Define number of output classes.
    num_classes = 7

    # Define padding scheme.
    padding = 'VALID'

    # Model Architecture
    network = input_data(shape=[None, 224, 224, 1])
    conv_1 = relu(conv_2d(network, 64, 7, strides=2, bias=True, padding=padding, activation=None, name='Conv2d_1'))
    maxpool_1 = batch_normalization(max_pool_2d(conv_1, 3, strides=2, padding=padding, name='MaxPool_1'))
    LRN_1 = local_response_normalization(maxpool_1, name='LRN_1')
    # FeatEX-1
    conv_2a = relu(conv_2d(maxpool_1, 96, 1, strides=1, padding=padding, name='Conv_2a_FX1'))
    maxpool_2a = max_pool_2d(maxpool_1, 3, strides=1, padding=padding, name='MaxPool_2a_FX1')
    conv_2b = relu(conv_2d(conv_2a, 208, 3, strides=1, padding=padding, name='Conv_2b_FX1'))
    conv_2c = relu(conv_2d(maxpool_2a, 64, 1, strides=1, padding=padding, name='Conv_2c_FX1'))
    FX1_out = merge([conv_2b, conv_2c], mode='concat', axis=3, name='FX1_out')
    # FeatEX-2
    conv_3a = relu(conv_2d(FX1_out, 96, 1, strides=1, padding=padding, name='Conv_3a_FX2'))
    maxpool_3a = max_pool_2d(FX1_out, 3, strides=1, padding=padding, name='MaxPool_3a_FX2')
    conv_3b = relu(conv_2d(conv_3a, 208, 3, strides=1, padding=padding, name='Conv_3b_FX2'))
    conv_3c = relu(conv_2d(maxpool_3a, 64, 1, strides=1, padding=padding, name='Conv_3c_FX2'))
    FX2_out = merge([conv_3b, conv_3c], mode='concat', axis=3, name='FX2_out')
    print(FX2_out)
    #added by Christiaan 
    GAP = global_avg_pool (FX2_out, name='GlobalAvgPool')
    
    net = flatten(FX2_out)
    if dropout:
        net = dropout(net, dropout_keep_prob)
    loss = fully_connected(net, num_classes,activation='softmax')

    # Compile the model and define the hyperparameters
    network = tflearn.regression(loss, optimizer='Adam',
                         loss='categorical_crossentropy',
                         learning_rate=0.0001)

Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.
Tensor("FX2_out/concat:0", shape=(?, 50, 50, 272), dtype=float32)
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [6]:


#create a custom tensorflow session to manage the used resources
# config = tf.ConfigProto()
# config.gpu_options.allow_growth = True
# session = tf.Session(config = config)


# Final definition of model checkpoints and other configurations
#model = tflearn.DNN(network, checkpoint_path='/home/cc/DeXpression/DeXpression_checkpoints',
# model = tflearn.DNN(network, checkpoint_path=tf_checkpoints,
#                     max_checkpoints=1, tensorboard_verbose=2, tensorboard_dir="./tflearn_logs/")


In [7]:
#create a custom tensorflow session to manage the used resources
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config = config)

In [8]:
# model = tflearn.DNN(network, checkpoint_path='./tf_checkpoints',
#                     max_checkpoints=1, tensorboard_verbose=2)
model = tflearn.DNN(network)


In [9]:
model.load('./tf_checkpoints/DeXpression_run_1.model')


INFO:tensorflow:Restoring parameters from G:\Documenten\personal\school\MaNaMA_AI\thesis\implementation\dexpression\github_1\github\Thesis_Christiaan_Jean\Custom_Dexpression\layer_visualisation\tf_checkpoints\DeXpression_run_1.model


In [13]:



# model1 = tflearn.DNN(FX2_out ,session=model.session)
model1 = tflearn.DNN(net ,session=model.session)
model2 = tflearn.DNN(FX2_out ,session=model.session)
model3 = tflearn.DNN(GAP ,session=model.session)
#NOTE there is no difference between using the net layer or FX2_out 


# model3 = tflearn.DNN(network, checkpoint_path=tf_checkpoints,
#                     max_checkpoints=1, tensorboard_verbose=2, tensorboard_dir="./tflearn_logs/")

# model4 = tflearn.DNN(network, checkpoint_path=tf_checkpoints,
#                     max_checkpoints=1, tensorboard_verbose=2, tensorboard_dir="./tflearn_logs/")

# model5 = tflearn.DNN(network, checkpoint_path=tf_checkpoints,
#                     max_checkpoints=1, tensorboard_verbose=2, tensorboard_dir="./tflearn_logs/")



# model1.load('./tf_checkpoints/DeXpression_run_1.model')
# model3.load('./tf_checkpoints/DeXpression_run_3.model')
# model4.load('./tf_checkpoints/DeXpression_run_4.model')
# model5.load('./tf_checkpoints/DeXpression_run_5.model')

In [14]:
# GAP_weights= model3.get_weights(GAP.W)
# showInfo(GAP_weights)

In [15]:
tf.RunOptions(report_tensor_allocations_upon_oom = True)
# classes = create_Y_pred_classes(divided_data,model1)
# print(classes[0])
# print(classes[1])

report_tensor_allocations_upon_oom: true

In [16]:
X_data[0:1].shape

display(X_data[0:1].astype('uint8'))


In [17]:
Y_real = model.predict(X_data[0:1])  # prediction
Y_pred = model1.predict(X_data[0:1])  # prediction
Y_pred2 = model2.predict(X_data[0:1])  # prediction
GAP_pred = model3.predict(X_data[0:1])  # prediction

In [19]:
showInfo(GAP_pred)
print(GAP_pred)

Name  UnKnown
type:  <class 'numpy.ndarray'>
Dtype:  float32
shape:  (1, 272)
[[7.09497929e-02 7.92344511e-02 3.26615982e-02 1.77221105e-01
  1.79613963e-01 3.62482481e-02 1.50022417e-01 7.13902339e-02
  2.40649171e-02 4.26748544e-02 8.33982043e-03 4.64239195e-02
  7.38788303e-03 1.76718652e-01 3.88964415e-02 1.89560913e-02
  8.31491873e-03 1.28904268e-01 1.97036102e-01 9.68137458e-02
  2.57805213e-02 1.72525898e-01 2.38229290e-01 7.01918006e-02
  3.16089168e-02 1.09849893e-01 8.73648562e-03 4.15991433e-03
  7.47172087e-02 1.31678343e-01 1.58947818e-02 9.96717345e-03
  1.64457902e-04 1.42886475e-01 2.28534326e-01 1.75502561e-02
  2.41901036e-02 1.47298396e-01 2.77230963e-02 5.58677875e-02
  1.28561646e-01 3.24806392e-01 2.11955104e-02 8.40060562e-02
  2.86389887e-02 9.61888283e-02 2.11796071e-02 1.99168152e-03
  2.40402557e-02 4.71307412e-02 2.40612738e-02 8.33149999e-02
  1.15752369e-02 0.00000000e+00 3.37164328e-02 3.05859023e-03
  4.38170135e-03 5.66708632e-02 1.29887059e-01 3.15484

In [12]:
# shows the images stored in the second to last layer
showInfo(Y_pred,"Y_pred after dropout")
showInfo(Y_pred2,"Y_pred2 before dropout")
maxo = np.max(Y_pred2)
print(maxo)
Y_reshape = cv2.resize(Y_pred2.reshape(50,50,272),(224,224), interpolation = cv2.INTER_AREA)
print(Y_reshape.shape[2])
showInfo(Y_reshape,'Y_reshape')
cv2.imshow("test",Y_reshape[:,:,240])
cv2.waitKey(0)


    
 

pred = Y_pred.reshape(1,50,50,272)
showInfo(pred,"pred = reshapen Y_pred")
np.set_printoptions(threshold=np.nan)
np.set_printoptions(suppress=True)
# print(Y_pred2[0,0,0,0:50])
# print("---------------------------")
# print(pred[0,0,0,0:50])

Name  Y_pred after dropout
type:  <class 'numpy.ndarray'>
Dtype:  float32
shape:  (1, 680000)
Name  Y_pred2 before dropout
type:  <class 'numpy.ndarray'>
Dtype:  float32
shape:  (1, 50, 50, 272)
5.647466
272
Name  Y_reshape
type:  <class 'numpy.ndarray'>
Dtype:  float32
shape:  (224, 224, 272)
Name  pred = reshapen Y_pred
type:  <class 'numpy.ndarray'>
Dtype:  float32
shape:  (1, 50, 50, 272)


In [13]:
# shows all images stored in the second to last layer
# for i in range(0,Y_reshape.shape[2]):
                 
# #     if(Y_reshape[:,:,i].any() >=  1):       
# # #         print(Y_reshape[:,:,1])
# #     print("max found")
#     str = "test " +  repr(i)
#     cv2.imshow(str,(Y_reshape[:,:,i]))
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()

KeyboardInterrupt: 

In [14]:
# compare the self calculated output with the real output
weights= model1.get_weights(loss.W)
print(type(weights))
showInfo(weights,'weights')
print(weights[:20][0])
print(weights[2500:2520][0])
print(weights[:20][1])
showInfo(np.exp(Y_pred))

elem = np.zeros((7,50,50))
elem_pos = np.zeros((7,50,50))
elem_pos_1 = np.zeros((7,50,50))
norm = np.zeros((7,50,50))
showInfo(elem)

for i in range(0,7):
    elem[i] = np.sum(np.multiply(Y_pred,weights[:,i]).reshape(50,50,272),axis = 2)
    elem_pos[i] = elem[i]-(np.min(elem))
#     elem_pos[i] = np.exp(elem[i]) # try the exponent
    norm[i] = elem_pos[i]/np.max(elem_pos) #bring weights between 0 and 1
    
    Y_norm = cv2.resize(norm[i],(224,224), interpolation = cv2.INTER_AREA)
    stre= "class " + repr(i)
    cv2.imshow(stre,Y_norm)
    cv2.waitKey(0)
    cv2.destroyAllWindows() 
#     for x in range(0,elem[i].shape[0]):
#         for y in range(0,elem[i].shape[1]):
#             if(elem[i][x][y] > 0 ):
#                 elem_pos_1[i][x][y] = elem[i][x][y]
#             else:
#                 elem_pos_1[i][x][y] = 0 
    
#     norm_1 = elem_pos[i]/np.max(elem_pos)
    
#     Y_norm_1 = cv2.resize(norm_1,(224,224), interpolation = cv2.INTER_AREA)
#     stre= "class " + repr(i)
#     cv2.imshow(stre,Y_norm_1)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows() 
    
#     showInfo(elem[i],'elem')



output =np.exp(np.sum(np.sum(elem,axis = 1),axis = 1) )  
# output =np.exp(np.matmul(Y_pred,weights))

showInfo(output,"output")
summ = np.sum(output)
print(output)
print(summ)
print("my output, ",  output/summ)
print("real output " ,Y_real)

<class 'numpy.ndarray'>
Name  weights
type:  <class 'numpy.ndarray'>
Dtype:  float32
shape:  (680000, 7)
[-0.02426779  0.01906147 -0.01809155 -0.00829517  0.00523452  0.03273747
 -0.00174737]
[ 0.02028005  0.02345897 -0.01292278  0.00358047  0.02017989  0.00263044
  0.01224747]
[ 0.01555516 -0.02866572 -0.00964126  0.00573142 -0.01376486 -0.02001086
  0.01070189]
Name  UnKnown
type:  <class 'numpy.ndarray'>
Dtype:  float32
shape:  (1, 680000)
Name  UnKnown
type:  <class 'numpy.ndarray'>
Dtype:  float64
shape:  (7, 50, 50)
Name  output
type:  <class 'numpy.ndarray'>
Dtype:  float64
shape:  (7,)
[   11.0337584      0.00512938 91236.31803717     0.13619357
     0.02390368     0.005165       0.0898027 ]
91247.61198990115
my output,  [0.00012092 0.00000006 0.99987623 0.00000149 0.00000026 0.00000006
 0.00000098]
real output  [[0.00012096 0.00000006 0.99987626 0.00000149 0.00000026 0.00000006
  0.00000098]]


In [ ]:
for i in range(0,7):
    elem_pos[i] = elem[i]-(np.min(elem))
#     elem_pos[i] = np.exp(elem[i]) # try the exponent
    norm[i] = elem_pos[i]/np.max(elem_pos) #bring weights between 0 and 1
    
    Y_norm = cv2.resize(norm[i],(224,224), interpolation = cv2.INTER_AREA)
    stre= "class " + repr(i)
    cv2.imshow(stre,Y_norm)
    cv2.waitKey(0)
    cv2.destroyAllWindows() 

In [ ]:
weight_class_0 = (weights.reshape(50,50,272,7))[:,:,:,0]
showInfo(weight_class_0)
maxi = np.max(weights)
print(maxi)

vis_weights = (weights.reshape(50,50,272,7))[:,:,0,0]
vis_224 = cv2.resize(vis_weights,(224,224), interpolation = cv2.INTER_AREA)
showInfo(vis_weights,'weights')

cv2.imshow("test",vis_weights)
cv2.waitKey(0)
print(vis_weights[0:10,0])

In [ ]:
# calculate the weight average of for 1 emotion
for i in range(2,7):
    multi = np.multiply((weights.reshape(50,50,272,7))[:,:,:,i],Y_pred.reshape(50,50,272))
    showInfo(multi,multi)
    Y_multi = cv2.resize(multi.reshape(50,50,272),(224,224), interpolation = cv2.INTER_AREA)
    
    
    for i in range(0,Y_reshape.shape[2]):

    #     if(Y_reshape[:,:,i].any() >=  1):       
    # #         print(Y_reshape[:,:,1])
#         print("max found")
        str = "test " +  repr(i)
        cv2.imshow(str,(Y_multi[:,:,i]))
        cv2.waitKey(0)
        cv2.destroyAllWindows()    
   
    
    sumi = np.sum(multi,axis=2)
    showInfo(sumi)
    print("why nut?")
    exp = np.exp(sumi)
    showInfo(exp,'exp')
    outp = exp/np.max(exp)
    
    
    sumi_vis_224 = cv2.resize(outp,(224,224), interpolation = cv2.INTER_AREA)
    
    
    
    stre= "class " + repr(i)
    cv2.imshow(stre,sumi_vis_224.astype('uint8'))
    cv2.waitKey(0)
